In [1]:
import sys, os

import pandas as pd

sys.path.insert(0, os.path.abspath('..'))
from src.train_mlm import get_data

In [2]:
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

with initialize(config_path="../conf"):
    cfg = compose(config_name="config.yaml")
    print(cfg)

dataset = get_data(cfg)

{'architecture': 'mlm', 'mlruns_dir': 'mlruns', 'overwrite_data_cache': True, 'data_path': 'data', 'data_subfolder': 'data/all_data_en', 'download_url': 'https://crisisnlp.qcri.org/data/crisis_datasets_benchmarks/crisis_datasets_benchmarks_v1.0.tar.gz', 'task': 'informativeness', 'language': 'EN', 'seed': 42, 'gpu': {'fp16': True, 'fp16_opt_level': 'O2', 'half_precision_backend': 'auto'}, 'debugging_mode': False, 'mode': {'name': 'train', 'do_train': True, 'do_eval': True, 'continue_training': False, 'per_device_train_batch_size': 32, 'per_device_eval_batch_size': 32, 'learning_rate': 3e-05, 'weight_decay': 0.1, 'num_train_epochs': 60, 'pad_to_max_length': False, 'max_seq_length': 'None', 'mlm_probability': 0.15, 'evaluation_strategy': 'steps', 'eval_steps': 500, 'logging_steps': 5000, 'early_stopping': True, 'patience': 50, 'save_model': True}, 'model': {'name': 'distilbert', 'pretrained_model': 'distilbert-base-uncased'}}


Reusing dataset crisis_bench_dataset (/export/home/kraft/.cache/huggingface/datasets/crisis_bench_dataset/informativeness/1.0.0/2e8a367b9209a9e88eb3aeca65dc626e3d123def2e180cfec317671f187fa562)


  0%|          | 0/3 [00:00<?, ?it/s]

Loaded dataset with DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 109796
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 16008
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 31095
    })
})


In [13]:
print(next(iter(dataset['test']['text'])))

Tesla finishes first of many solar projects in Puerto Rico Read more : http http


In [5]:
test_set_path = "../data/data/all_data_en/crisis_consolidated_informativeness_filtered_lang_en_test.tsv"
delim = "\t"

In [35]:
from src.TweetNormalizer import normalizeTweet

texts = []
with open(test_set_path, "r", newline=None, encoding='utf-8', errors='replace') as f:
    next(f)
    for i, line in enumerate(f):
        if i > 20:
            break
        line = line.strip()
        if line == "":
            continue
        row = line.split(delim)
        text = row[3].strip()
        # text = normalizeTweet(text)
        label = row[6]
        print(text, label)
        if label == "not_informative":
            continue
        texts.append(text)

texts

Tesla finishes first of many solar projects in Puerto Rico Read more: https://t.co/YOeeecGl7D https://t.co/FD1dYnGhDu not_informative
@LilyUnaSmith I'm like 2 miles away. We always get the day off for the marathon. I can see the area from my window. not_informative
RT @NBCSN: BOSTON MARATHON EXPLOSIONS: 1st explosion reported on North side of Boylston Street by the the finish line. 2nd explosion sec ... informative
Make that 5 co-workers thanks to Bailee. not_informative
BRING ALL YOUR WORRIES TO HIM BECAUSE HE CARES FOR YOU ���  #AyokoNaSana #ThingsToDoPagMalamig #ForevermoreTheGrandeBash #RubyPH not_informative
"#Intern #Accounting Internship (Finance and Economics) - San Antonio, TX Job: Scottrade - S... http://t.co/wPlYA03WBK #Texas #Internship" not_informative
My wish for you is that this life becomes all that you want it to not_informative
Loan Upheaval Is The Way In Which Oneself Can Save Your House Leaving out Being Foreclosed On...TEJc not_informative
Spring in Paris be like @

['RT @NBCSN: BOSTON MARATHON EXPLOSIONS: 1st explosion reported on North side of Boylston Street by the the finish line. 2nd explosion sec ...',
 '.@UNOSAT: Over 15 million people exposed to strong shaking after #Mexicoearthquake. Report: https://t.co/HmlLW2iXuY https://t.co/MgHlcu6zPQ',
 'Update: Vanuatu Hit Hard by Cyclone http://t.co/uss9ByNnSe',
 'RT @911BUFF: BOSTON: FBI CONFIRMS 2ND SUSPECT ON THE LOOSE! ADDITIONAL ARMORED VEHICLES, BOMB SQUAD AND HOMELAND SECURITY HEADING TO THE SC…',
 'RT @timmytink: Colorado wildfire: Extreme weather conditions stoke High Park Fire, tax firefighters http://t.co/TIIaLAVY via @denverpost',
 "RT @lisabelkin: You couldn't make this stuff up: Runner witnesses Marathon explosion, returns to TX and sees fertilizer explosion. http: ..."]

In [17]:
from transformers import AutoTokenizer, DataCollatorForWholeWordMask
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", normalize=True)
tokenized_texts = [tokenizer(txt,
                         padding=True,
                         truncation=True, max_length=None) for txt in texts]
data_collator = DataCollatorForWholeWordMask(tokenizer, 0.15)
batched = data_collator(tokenized_texts)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
batched

11

In [21]:
from transformers import pipeline

mask_filler = pipeline("fill-mask", model="vinai/bertweet-base")
mask_filler(batched['input_ids'])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [38]:
mask_filler("RT @NBCSN: BOSTON MARATHON EXPLOSIONS: 1st <mask> reported on North side of "
            "Boylston Street by the the finish line. 2nd explosion sec ...")

[{'score': 0.9371997714042664,
  'token': 15839,
  'token_str': 'e x p l o s i o n',
  'sequence': 'RT @NBCSN: BOSTON MARATHON EXPLOSIONS: 1st explosion reported on North side of Boylston Street by the the finish line. 2nd explosion sec...'},
 {'score': 0.01401183195412159,
  'token': 4472,
  'token_str': 'b l a s t',
  'sequence': 'RT @NBCSN: BOSTON MARATHON EXPLOSIONS: 1st blast reported on North side of Boylston Street by the the finish line. 2nd explosion sec...'},
 {'score': 0.01373971812427044,
  'token': 39021,
  'token_str': 'e x p l o s i o n s',
  'sequence': 'RT @NBCSN: BOSTON MARATHON EXPLOSIONS: 1st explosions reported on North side of Boylston Street by the the finish line. 2nd explosion sec...'},
 {'score': 0.006707529537379742,
  'token': 39566,
  'token_str': 'E x p l o s i o n',
  'sequence': 'RT @NBCSN: BOSTON MARATHON EXPLOSIONS: 1st Explosion reported on North side of Boylston Street by the the finish line. 2nd explosion sec...'},
 {'score': 0.004978061653673649,
 